# Fonction de Rosenbrock généralisée

In [1]:
using LinearAlgebra
using ForwardDiff
using BenchmarkTools

Plusieurs généralisations à $n$ dimensions de la fonction de Rosenbrock ont été proposées dans la littérature. Nous considérons ici la généralisation définie:
$$
f(x) = \sum_{i = 1}^{n-1} \left( 100(x_{(i+1)}^2-x_{(i)})^2 + (x_{(i)}-1)^2 \right).
$$
Il s'agit de la somme de $n-1$ fonctions de Rosenbrock à 2 dimensions.

In [2]:
function rosenbrock(x::Vector)
    return sum(100*(x[i+1]^2 - x[i])^2 + (x[i] - 1)^2 for i in 1:n-1)
end

rosenbrock (generic function with 1 method)

Le gradient est
$$
\nabla_x f(x) =
\begin{pmatrix}
-200(x_{(2)}^2-x_{(1)})+2(x_{(1)}-1) \\
400x_{(2)}(x_{(2)}^2-x_{(1)})-200(x_{(3)}^2-x_{(2)})+2(x_{(2)}-1) \\
\vdots \\
400x_{(i)}(x_{(i)}^2-x_{(i-1)})-200(x_{(i+1)}^2-x_{(i)})+2(x_{(i)}-1) \\
\vdots \\
400x_{(n-1)}(x_{(n-1)}^2-x_{(n-2)})-200(x_{(n-1)}^2-x_{(n-2)})+2(x_{(n-2)}-1) \\
400x_{(n)}(x_{(n)}^2-x_{(n-1)})
\end{pmatrix}
$$

In [3]:
function ∇f(x:: Vector)
    n = length(x)
    g = zeros(n)
    for i = 1:n-1
        g[i] = -200*(x[i+1]^2-x[i])+2*(x[i]-1)
    end
    for i = 2:n
        g[i] += 400*x[i]*(x[i]^2-x[i-1])
    end
    return g
end

∇f (generic function with 1 method)

La matrice hessienne est
$$
\nabla_{xx}^2 f(x) =
\begin{pmatrix}
202 & -400x_{(2)} & 0 & 0 & \cdots & 0 & 0 & 0 \\
 -400x_{(2)} & 400(x_{(2)}^2-x(1))+800x_{(2)}^2+202 & -400x_{(3)} & 0 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots &\vdots &\vdots & \ddots & \vdots &\vdots &\vdots &\\
0 & 0 & 0 & 0 & \cdots & -400x_{(n-2)} & 400(x_{(n-1)}^2-x_{(n-2)})+800x_{(n-1)}^2 +202 & -400x_{(n-1)} \\
0 & 0 & 0 & 0 & \cdots & 0 & -400x_{(n-1)} & 400(x_{(n)}^2-x_{(n-1)})+800x_{(n)}^2
\end{pmatrix}
$$

In [4]:
function Hess(x:: Vector)
    n = length(x)
    H = zeros(n,n)
    H[1,1] = 202
    for i = 2:n
        H[i,i-1] = H[i-1,i] = -400*x[i]
        H[i,i] = 400*(x[i]^2-x[i-1])+800*x[i]^2 + 202
    end
    H[n,n] -= 202
    return H
end

Hess (generic function with 1 method)

Les calculs sont complexes! Nous allons les vérifier à l'aide de la différentiation automatique.

In [5]:
g = x -> ForwardDiff.gradient(rosenbrock, x);
H = x -> ForwardDiff.hessian(rosenbrock, x);
function g!(storage::Vector, x::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end
function H!(storage::Matrix, x::Vector)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [6]:
n = 1000
x = ones(n)/4

1000-element Array{Float64,1}:
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 ⋮
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25

In [7]:
rosenbrock(x)

4074.046875

In [8]:
norm(g(x)-∇f(x))

0.0

In [9]:
norm(H(x)-Hess(x))

0.0

In [10]:
H(x)

1000×1000 Array{Float64,2}:
  202.0  -100.0     0.0     0.0     0.0  …     0.0     0.0     0.0     0.0
 -100.0   177.0  -100.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0  -100.0   177.0  -100.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0  -100.0   177.0  -100.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0  -100.0   177.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0  -100.0  …     0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0  …     0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0

La différentiation automatique est-elle efficace?

In [11]:
@benchmark H(x)

BenchmarkTools.Trial: 
  memory estimate:  27.22 MiB
  allocs estimate:  39030
  --------------
  minimum time:     9.675 s (0.00% GC)
  median time:      9.675 s (0.00% GC)
  mean time:        9.675 s (0.00% GC)
  maximum time:     9.675 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [12]:
@benchmark Hess(x)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     7.164 ms (0.00% GC)
  median time:      11.196 ms (0.00% GC)
  mean time:        13.441 ms (14.67% GC)
  maximum time:     39.476 ms (34.45% GC)
  --------------
  samples:          372
  evals/sample:     1

On voit qu'il y a avantage a utiliser l'implémentation exacte de la matrice hessienne, mais est-ce dû au caractère creux de la matrice?

In [13]:
using SparseArrays

In [17]:
He = Hess(x)
SH = sparse(He)

1000×1000 SparseMatrixCSC{Float64,Int64} with 2998 stored entries:
  [1   ,    1]  =  202.0
  [2   ,    1]  =  -100.0
  [1   ,    2]  =  -100.0
  [2   ,    2]  =  177.0
  [3   ,    2]  =  -100.0
  [2   ,    3]  =  -100.0
  [3   ,    3]  =  177.0
  [4   ,    3]  =  -100.0
  [3   ,    4]  =  -100.0
  [4   ,    4]  =  177.0
  [5   ,    4]  =  -100.0
  [4   ,    5]  =  -100.0
  ⋮
  [996 ,  996]  =  177.0
  [997 ,  996]  =  -100.0
  [996 ,  997]  =  -100.0
  [997 ,  997]  =  177.0
  [998 ,  997]  =  -100.0
  [997 ,  998]  =  -100.0
  [998 ,  998]  =  177.0
  [999 ,  998]  =  -100.0
  [998 ,  999]  =  -100.0
  [999 ,  999]  =  177.0
  [1000,  999]  =  -100.0
  [999 , 1000]  =  -100.0
  [1000, 1000]  =  -25.0

Comme ce qui nous intéresse est le produit matrice-vecteur, regarde le gain potentiel obtenu avec le stockage creux.

In [18]:
@benchmark SH*x

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     6.333 μs (0.00% GC)
  median time:      12.566 μs (0.00% GC)
  mean time:        26.091 μs (12.25% GC)
  maximum time:     9.804 ms (99.69% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [19]:
@benchmark He*x

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     453.501 μs (0.00% GC)
  median time:      811.600 μs (0.00% GC)
  mean time:        1.015 ms (0.00% GC)
  maximum time:     23.216 ms (0.00% GC)
  --------------
  samples:          4853
  evals/sample:     1

Plutôt que de construire la matrice dense puis de la réduire à une matrice creuse, nous pouvons directement utiliser les propriétés de la matrice pour exploiter sa structure, par exemple en exploitant le fait qu'il s'agit d'une matrice tridiagonale symétrique.

In [20]:
function TriHess(x)
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl)
    return H
end

TriHess (generic function with 1 method)

In [21]:
T = TriHess(x)
@benchmark T*x

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     1.540 μs (0.00% GC)
  median time:      5.680 μs (0.00% GC)
  mean time:        11.672 μs (16.66% GC)
  maximum time:     2.239 ms (98.61% GC)
  --------------
  samples:          10000
  evals/sample:     10

Cela permet de monter la taille du système, qui sinon serait ingérable. Prenons par exemple une dimension de un million.

In [22]:
x = ones(1000000)/4
T = TriHess(x)
@benchmark T*x

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     6.635 ms (0.00% GC)
  median time:      9.743 ms (0.00% GC)
  mean time:        11.225 ms (11.88% GC)
  maximum time:     26.944 ms (54.41% GC)
  --------------
  samples:          445
  evals/sample:     1

Revenons à un exemple plus modeste.

In [23]:
x = ones(1000)/4
v = copy(x)

1000-element Array{Float64,1}:
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 ⋮
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25

In [24]:
rosenbrock(x)

4074.046875

In [25]:
gvr = x -> dot(g(x), v)
@benchmark gvr(x)

BenchmarkTools.Trial: 
  memory estimate:  128.45 KiB
  allocs estimate:  467
  --------------
  minimum time:     1.442 ms (0.00% GC)
  median time:      1.780 ms (0.00% GC)
  mean time:        2.091 ms (0.91% GC)
  maximum time:     13.899 ms (68.10% GC)
  --------------
  samples:          2386
  evals/sample:     1

In [26]:
function gvr2(x:: Vector)
    n = length(x)
    t = (-200*(x[2]^2-x[1])+2*(x[1]-1))*v[1]
    for i = 2:n-1
        t += (-200*(x[i+1]^2-x[i])+2*(x[i]-1)+400*x[i]*(x[i]^2-x[i-1]))*v[i]
    end
    t += 400*x[n]*(x[n]^2-x[n-1])*v[n]
    return t
end
@benchmark gvr2(x)

BenchmarkTools.Trial: 
  memory estimate:  70.13 KiB
  allocs estimate:  4488
  --------------
  minimum time:     118.100 μs (0.00% GC)
  median time:      134.700 μs (0.00% GC)
  mean time:        212.960 μs (0.90% GC)
  maximum time:     6.572 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [27]:
Hv = x -> ForwardDiff.gradient(gvr, x)
norm(Hv(x)-TriHess(x)*v)

0.0

In [28]:
Hv2 = x -> ForwardDiff.gradient(gvr2, x)
norm(Hv2(x)-TriHess(x)*v)

0.0

Comparons les temps de calcul des différentes approches.

In [29]:
@benchmark TriHess(x)*v

BenchmarkTools.Trial: 
  memory estimate:  31.78 KiB
  allocs estimate:  5
  --------------
  minimum time:     5.500 μs (0.00% GC)
  median time:      7.280 μs (0.00% GC)
  mean time:        11.275 μs (12.97% GC)
  maximum time:     1.629 ms (99.32% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [30]:
@benchmark Hess(x)*v

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  3
  --------------
  minimum time:     5.507 ms (0.00% GC)
  median time:      7.425 ms (0.00% GC)
  mean time:        8.927 ms (15.79% GC)
  maximum time:     28.265 ms (73.41% GC)
  --------------
  samples:          560
  evals/sample:     1

In [31]:
@benchmark Hv(x)

BenchmarkTools.Trial: 
  memory estimate:  127.83 MiB
  allocs estimate:  39358
  --------------
  minimum time:     6.118 s (0.24% GC)
  median time:      6.118 s (0.24% GC)
  mean time:        6.118 s (0.24% GC)
  maximum time:     6.118 s (0.24% GC)
  --------------
  samples:          1
  evals/sample:     1

In [32]:
@benchmark Hv2(x)

BenchmarkTools.Trial: 
  memory estimate:  28.92 MiB
  allocs estimate:  377038
  --------------
  minimum time:     20.306 ms (0.00% GC)
  median time:      31.614 ms (5.63% GC)
  mean time:        33.478 ms (4.24% GC)
  maximum time:     81.287 ms (3.50% GC)
  --------------
  samples:          150
  evals/sample:     1

On voit qu'exploiter la structure de la matrice est plus efficace! Une grande part de la sous-performance vient de la nom prise en compte du caractère creux par la différentiation automatique, et on gagne à implémenter des calculs exacts.

## Inversion de matrices creuses

Si la matrice hessienne est creuse, son inverse peut être dense. Il n'est dès lors pas pratique d'utiliser des méthodes basées sur l'inverse de la matrice hessienne.

In [33]:
H = TriHess(x)

1000×1000 SymTridiagonal{Float64,Array{Float64,1}}:
  202.0  -100.0      ⋅       ⋅       ⋅   …      ⋅       ⋅       ⋅       ⋅ 
 -100.0   177.0  -100.0      ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅   -100.0   177.0  -100.0      ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅   -100.0   177.0  -100.0         ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅   -100.0   177.0         ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅   -100.0  …      ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅   …      ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          

In [34]:
inv(H)

1000×1000 Array{Float64,2}:
 -0.235182   -0.485068  -0.623388  …   0.350157    0.0606859  -0.242743
 -0.485068   -0.979838  -1.25924       0.707318    0.122585   -0.490342
 -0.623388   -1.25924   -1.60547       0.901796    0.15629    -0.625162
 -0.618329   -1.24903   -1.59245       0.88886     0.154049   -0.616194
 -0.471054   -0.95153   -1.21315       0.671487    0.116376   -0.465502
 -0.215437   -0.435183  -0.554837  …   0.299671    0.0519361  -0.207745
  0.0897308   0.181256   0.231093     -0.141068   -0.0244486   0.0977943
  0.374261    0.756006   0.963871     -0.549362   -0.0952101   0.38084
  0.57271     1.15688    1.47496      -0.831303   -0.144073    0.576293
  0.639437    1.29166    1.64681      -0.922044   -0.1598      0.639199
  0.559093    1.12937    1.43989   …  -0.800715   -0.138772    0.555088
  0.350157    0.707318   0.901796     -0.495221   -0.0858269   0.343308
  0.0606859   0.122585   0.15629      -0.0758269  -0.0131416   0.0525663
  ⋮                                

Factoriser la matrice n'implique pas le même niveau de remplissage. Dans le cas présent, il est remarquable de noter que la structure creuse est parfaitement respectée.

In [35]:
factorize(H)

LDLt{Float64,SymTridiagonal{Float64,Array{Float64,1}}}
L factor:
1000×1000 UnitLowerTriangular{Float64,SymTridiagonal{Float64,Array{Float64,1}}}:
  1.0        ⋅          ⋅       …    ⋅         ⋅        ⋅         ⋅ 
 -0.49505   1.0         ⋅            ⋅         ⋅        ⋅         ⋅ 
  0.0      -0.784344   1.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0       -1.01455       ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0      …    ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0      …    ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0           ⋅         ⋅        ⋅         ⋅ 
  0.0       0.0        0.0

Cela permet à nouveau de privilégier la résolutions de systèmes linéaires par factorisation à l'inversion de matrice, mais l'effort de calcul pour la factorisation reste significatif.

In [36]:
@benchmark factorize(H)

BenchmarkTools.Trial: 
  memory estimate:  15.91 KiB
  allocs estimate:  3
  --------------
  minimum time:     12.600 μs (0.00% GC)
  median time:      13.699 μs (0.00% GC)
  mean time:        17.592 μs (4.38% GC)
  maximum time:     3.401 ms (99.12% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [37]:
@benchmark H\v

BenchmarkTools.Trial: 
  memory estimate:  23.81 KiB
  allocs estimate:  3
  --------------
  minimum time:     20.299 μs (0.00% GC)
  median time:      22.301 μs (0.00% GC)
  mean time:        29.287 μs (4.01% GC)
  maximum time:     3.196 ms (98.50% GC)
  --------------
  samples:          10000
  evals/sample:     1

Malheureusement, tous les problèmes ne présentent pas une structure aussi intéressante. Néanmoins, l'implémentation directe du produit matrice-vecteur permet de contourner un autre problème: le stockage des matrices en grande dimension. Reprenons le stockage dense. On a

In [38]:
@benchmark Hess(x)\v

BenchmarkTools.Trial: 
  memory estimate:  15.27 MiB
  allocs estimate:  6
  --------------
  minimum time:     32.243 ms (0.00% GC)
  median time:      43.500 ms (0.00% GC)
  mean time:        47.418 ms (5.48% GC)
  maximum time:     136.465 ms (0.00% GC)
  --------------
  samples:          106
  evals/sample:     1

In [39]:
x = ones(100000)/4
v = copy(x)

100000-element Array{Float64,1}:
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 ⋮
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25
 0.25

In [40]:
@benchmark Hess(x)\v

LoadError: OutOfMemoryError()

In [41]:
@benchmark TriHess(x)\v

BenchmarkTools.Trial: 
  memory estimate:  4.58 MiB
  allocs estimate:  13
  --------------
  minimum time:     2.608 ms (0.00% GC)
  median time:      3.552 ms (0.00% GC)
  mean time:        4.582 ms (6.16% GC)
  maximum time:     21.029 ms (19.69% GC)
  --------------
  samples:          1089
  evals/sample:     1

In [42]:
@benchmark Hv2(x)

BenchmarkTools.Trial: 
  memory estimate:  285.57 GiB
  allocs estimate:  4162741289
  --------------
  minimum time:     455.704 s (4.84% GC)
  median time:      455.704 s (4.84% GC)
  mean time:        455.704 s (4.84% GC)
  maximum time:     455.704 s (4.84% GC)
  --------------
  samples:          1
  evals/sample:     1

Lorsque le nombre de variables devient trop grand, la résolution directe du système linéaire avec une matrice dense est prohibitive (le code a pu résulter sur une erreur de débordement mémoire), alors qu'il est raisonnable de calculer le produit matrice-vecteur pour quelques itérations, ouvrant la voie aux méthodes de gradient conjugué tronqués. Exploiter la structure reste cependant à nouveau plus efficace.